In [1]:
import os
import itertools    
import cv2
import pickle
import keras
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from keras.layers import Activation, LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.utils import np_utils
from utils_PK import fetch_data_files
import numpy as np
import pandas as pd
import scipy.misc
import numpy.random as rng
import random
from PIL import Image, ImageDraw, ImageFont
from sklearn.utils import shuffle
import nibabel as nib #reading MR images
from sklearn.cross_validation import train_test_split
import math
import glob
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

C:\Users\pruthvi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\pruthvi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[[22. 28.]
 [49. 64.]]


In [2]:
# Save as Pickle file

DATA_PATH = 'E:\\005Balgrist\\009Longitudinal_prediction\\Data\\Patients' #% where the data is organized
lst = os.listdir(DATA_PATH)
if '.DS_Store' in lst:
    lst.remove('.DS_Store')

n_subjects=len(lst)
Time_points_list=["month_06","month_12","month_24"]

Modality_list=["MT","R1","R2s","mhR1","T1w","PDw","MTw","A","MTR_synt"]

# Label_integer_list=[50,70,70,36,42,46,35,36,43,27,34,39,38,42,40,81,80,68,35,31,20,100,100,100,56,56,36,21,30,25,96,91,96]
# Label_string_list=["50","70","70","36","42","46","35","36","43","27","34","39","38","42","40","81","80","68","35","31","20","100","100","100","56","56","36","21","30","25","96","91","96"]

Label_integer_list_rounded=[50,70,70,40,40,50,40,40,40,30,30,40,40,40,40,80,80,70,40,30,20,100,100,100,60,60,40,20,30,30,100,90,100]
Label_string_list_rounded=["50","70","70","40","40","50","40","40","40","30","30","40","40","40","40","80","80","70","40","30","20","100","100","100","60","60","40","20","30","30","100","90","100"]

n_time=len(Time_points_list)
n_Modality=len(Modality_list)
n_Label=len(Label_string_list_rounded)

Subjects_list1=list(itertools.chain.from_iterable((itertools.repeat(i, n_time*n_Modality) for i in lst)))

Time_points_list1=list(itertools.chain.from_iterable((itertools.repeat(i, n_Modality) for i in Time_points_list)))
Time_points_list2=list(itertools.chain.from_iterable(itertools.repeat(Time_points_list1, n_subjects)))

Modality_list1=list(itertools.chain.from_iterable(itertools.repeat(Modality_list, n_time*n_subjects)))
Label_list1=list(itertools.chain.from_iterable((itertools.repeat(i, n_Modality) for i in Label_string_list_rounded)))
Label_list2=list(itertools.chain.from_iterable((itertools.repeat(i, n_Modality) for i in Label_integer_list_rounded)))

print(len(Subjects_list1))
print(len(Time_points_list2))
print(len(Modality_list1))

# print((Subjects_list1))
# print((Time_points_list2))
# print((Modality_list1))
# print(Label_list2)


297
297
297


In [3]:
original_df = pd.DataFrame({"subject": Subjects_list1, "Time_point": Time_points_list2, "Modality": Modality_list1, "Labels": Label_list2})
original_df.to_pickle("E:\\005Balgrist\\009Longitudinal_prediction\\Data\\subject_dict.pkl")

# Just to open and see -- read python dict back from the file
pkl_file = open('E:\\005Balgrist\\009Longitudinal_prediction\\Data\\subject_dict.pkl', 'rb')
mydict2 = pickle.load(pkl_file)
pkl_file.close()

print(mydict2)

      subject Time_point  Modality  Labels
0    subj1032   month_06        MT      50
1    subj1032   month_06        R1      50
2    subj1032   month_06       R2s      50
3    subj1032   month_06      mhR1      50
4    subj1032   month_06       T1w      50
5    subj1032   month_06       PDw      50
6    subj1032   month_06       MTw      50
7    subj1032   month_06         A      50
8    subj1032   month_06  MTR_synt      50
9    subj1032   month_12        MT      70
10   subj1032   month_12        R1      70
11   subj1032   month_12       R2s      70
12   subj1032   month_12      mhR1      70
13   subj1032   month_12       T1w      70
14   subj1032   month_12       PDw      70
15   subj1032   month_12       MTw      70
16   subj1032   month_12         A      70
17   subj1032   month_12  MTR_synt      70
18   subj1032   month_24        MT      70
19   subj1032   month_24        R1      70
20   subj1032   month_24       R2s      70
21   subj1032   month_24      mhR1      70
22   subj10

In [4]:
# config['data_dict'] = pickle file containing a dictionary with at least the following keys: subject and contrast_foldname
# This dict is load as a panda dataframe and used by the function utils.fetch_data_files
# IMPORTANT NOTE: the testing dataset is not included in this dataframe
DATA_PD = pd.read_pickle("E:\\005Balgrist\\009Longitudinal_prediction\\Data\\subject_dict.pkl")
# print(DATA_PD)
DATA_FOLD = "E:\\005Balgrist\\009Longitudinal_prediction\\Data\\Patients"  # where the preprocess data are stored
MODEL_FOLD = "E:\\005Balgrist\\009Longitudinal_prediction\\Trained_models\\"  # where to store the trained models

In [5]:
# len_train = int(0.8 * len(DATA_PD.index)) # 80% of the dataset is used for the training 
# idx_train = random.sample(range(len(DATA_PD.index)), len_train)
# idx_valid = [ii for ii in range(len(DATA_PD.index)) if ii not in idx_train] # the remaining images are used for the validatio

len_Whole = int(1 * len(DATA_PD.index)) # 100% of the dataset is used for the training 
idx_Whole = random.sample(range(len(DATA_PD.index)), len_Whole)

In [6]:
# print(DATA_PD[DATA_PD.index.isin(idx_train)])

# training_files = fetch_data_files(data_frame=DATA_PD[DATA_PD.index.isin(idx_train)],
#                                   data_fold=DATA_FOLD)
# validation_files = fetch_data_files(data_frame=DATA_PD[DATA_PD.index.isin(idx_valid)],
#                                   data_fold=DATA_FOLD)

WHOLE_files = fetch_data_files(data_frame=DATA_PD[DATA_PD.index.isin(idx_Whole)],data_fold=DATA_FOLD)

# print(training_files)

In [7]:
# Training data
# images = []
# print(training_files[0][0])
# print(len(training_files[:][:]))

# for f in range(len(training_files)):
#     a = nib.load(training_files[f][0])
#     a = a.get_data()  
#     print(a.shape[0])
# #     a = a[:,:,:]
#     for i in range(a.shape[2]):
#         images.append((a[:,:,i]))
# print (a.shape)
# print(len(images))

# Whole data

In [8]:
images = []
# print(WHOLE_files[0][0])
# print(WHOLE_files[0][1])
# print(len(WHOLE_files[:][:]))
labels=[]
for f in range(len(WHOLE_files)):
    a = nib.load(WHOLE_files[f][0])
    a = a.get_data()  
#     print(a.shape[0])
#     a = a[:,:,:]
    for i in range(a.shape[2]):
        images.append((a[:,:,i]))
        b = WHOLE_files[f][1]
        labels.append(b)
print (a.shape)
print(len(images))
# print(labels)

(240, 256, 176)
52249


In [9]:
a[:,:,0].shape


(240, 256)

In [10]:
images = np.asarray(images)


In [11]:
images.shape

(52249, 240, 256)

In [12]:
images = images.reshape(-1, 240,256,1)


In [13]:
images.shape

(52249, 240, 256, 1)

In [14]:
m = np.max(images)
mi = np.min(images)
m,mi

(100000.0, -100000.0)

In [15]:
images = (images - mi) / (m - mi)


In [16]:
np.min(images), np.max(images)


(0.0, 1.0)

In [17]:
len(labels)

52249

In [18]:
temp = np.zeros([52249,256,256,1])
temp[:,16:,:,:] = images
images=temp


In [19]:
images.shape

(52249, 256, 256, 1)

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(images,labels,test_size=0.2,random_state=13)

In [21]:
# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([x_train,x_valid,y_train,y_valid], f,protocol=1)

# Getting back the objects:
with open('objs.pkl','rb') as f:  # Python 3: open(..., 'rb')
    x_train,x_valid,y_train,y_valid = pickle.load(f)

MemoryError: 

In [ ]:
plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121)
curr_img = np.reshape(x_train[0], (256,256))
plt.imshow(curr_img, cmap='gray')

# Display the first image in testing data
plt.subplot(122)
curr_img = np.reshape(x_valid[0], (256,256))
plt.imshow(curr_img, cmap='gray')

In [50]:
batch_size = 5
epochs = 50
inChannel = 1
x, y = 240, 256
img_rows, img_cols = 240, 256
input_shape=(x, y, inChannel)
input_img = Input(shape = (x, y, inChannel))

In [51]:
def autoencoder(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)


    #decoder
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 128
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    up1 = UpSampling2D((2,2))(conv4) # 14 x 14 x 128
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 64
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    up2 = UpSampling2D((2,2))(conv5) # 28 x 28 x 64
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_valid, y_valid))
score = model.evaluate(x_valid, y_valid, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [52]:
autoencoder = Model(input_img, autoencoder(input_img))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_1/MaxPool' (op: 'MaxPool') with input shapes: [?,32,256,1].

In [ ]:
autoencoder.summary()


In [ ]:
autoencoder_train = autoencoder.fit(train_X, train_ground, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_ground))


In [ ]:
loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(50)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
autoencoder = autoencoder.save_weights('autoencoder_mri.h5')
